KY16 pr måned. antal personer område. Alle ydelsestyper. fra og med 2007 til dato
Kontanthjælp (personer og beløb) efter område, enhed og ydelsestype (2007M01-2023M03)  
KONT1X: Kontanthjælpsmodtagere (månedlig, personer) efter område og ydelsesart (AFSLUTTET) 1994-2006
Se også kontanthjælp.py


In [65]:
import pandas as pd   

# Retrieving data

KY034 = "https://api.statbank.dk/v1/data/KY034/CSV?valuePresentation=CodeAndValue&TAL=1%2C2&OMR%C3%85DE=*&Tid=*"
    
KONT1X = "https://api.statbank.dk/v1/data/KONT1X/CSV?valuePresentation=CodeAndValue&OMR%C3%85DE=*&YDELSESART=TOT&Tid=*"

KY034_data = pd.read_csv(KY034, sep = ";")
KONT1X_data = pd.read_csv(KONT1X, sep = ";")    

In [66]:
def clean_ds_data(df):
    df['muni_code'] = df['OMRÅDE'].str.extract(r'(\d{3})')   # extracting municipality code
    df['muni_name'] = df['OMRÅDE'].str.extract(r'(\d{3}\s+(.*))')[1] # extracting municipalit name
    df['year'] = df['TID'].str.split().str[0]
    df = df\
        .drop(columns=["OMRÅDE", 'TID'])
    return df

In [67]:
# cleaning KONT1X data. Only looking at data for january. Remember to remove november!
KONT1X_data = clean_ds_data(KONT1X_data)
KONT1X_data = KONT1X_data[KONT1X_data['year'].str.endswith('01')].copy()
KONT1X_data['year'] = KONT1X_data['year'].str.replace('M01', "")

In [68]:
# Reshaping kont1x data


KONT1X_data = KONT1X_data.pivot(index=["muni_code", "year"], columns = "YDELSESART", values = "INDHOLD")
KONT1X_data.name = None
KONT1X_data = KONT1X_data.reset_index()

In [69]:
nye_gamle = pd.read_csv("../nye_og_gamle_kommuner.csv")
nye_gamle["muni_code"] = nye_gamle["muni_code"].astype("Int64")
KONT1X_data = KONT1X_data.rename(columns={"muni_code":"old_muni_code"})
KONT1X_data["old_muni_code"] = KONT1X_data["old_muni_code"].astype("Int64")
KONT1X_data = KONT1X_data.merge(nye_gamle, how='left')
KONT1X_data = KONT1X_data.groupby(["muni_code", "year"])["TOT Modtagere af kontanthjælp mv. i alt"].agg("sum").to_frame().reset_index()

In [70]:
# reshaping KY034 data

KY034_data = KY034_data.pivot(index=["OMRÅDE", "TID"], columns = "TAL", values = "INDHOLD").reset_index()
KY034_data = clean_ds_data(KY034_data)

In [71]:
# selecting and renaming columns.
KY034_data = KY034_data[["muni_code", "year", "1 Berørte personer (antal)"]].rename(columns={"1 Berørte personer (antal)":"kont_recip_tot"})
KONT1X_data = KONT1X_data[["muni_code", "year", "TOT Modtagere af kontanthjælp mv. i alt"]].rename(columns = {"TOT Modtagere af kontanthjælp mv. i alt":"kont_recip_tot"})

In [72]:
# coercing muni_code and year to integers
KY034_data['muni_code'] = KY034_data['muni_code'].astype('Int64')
KONT1X_data['muni_code'] = KONT1X_data['muni_code'].astype('Int64')

KY034_data['year'] = KY034_data['year'].astype('Int64')
KONT1X_data['year'] = KONT1X_data['year'].astype('Int64')

In [73]:
# KY034_data originally contains data on "landsdele". They appear as NaN in our data. Get rid of those!
KY034_data = KY034_data.dropna(subset=['muni_code'])


In [74]:
total = pd.concat([KY034_data, KONT1X_data]).sort_values(["muni_code", "year"])

In [75]:
total.to_csv("kontanth.csv")